# T1-Mapping with DESPOT1 / VFA

This notebook deomstrates how T1 mapping with a basic `nipype` pipeline with `QUIT` programs.

This pipeline (http://github.com/spinicist/nanslice) downloads the BrainWeb brain & B1 phantoms from http://brainweb.bic.mni.mcgill.ca. It then replaces the tissue classification labels with values of Proton Density and T1, simulates an SPGR/FLASH image with some added noise, and finally uses that simulated data to fit for T1 and PD again.

In order to display the images, and to do some 

## Imports

In [ ]:
%matplotlib inline
from QUIT.interfaces.relax import DESPOT1, DESPOT1Sim, DESPOT2
from nanslice import Layer
import nanslice.jupyter as ns
import nibabel as nib
import numpy as np
import requests
import gzip
import os.path

In [ ]:
## Create Phantom Data

This section downloads the Brainweb phantoms. These are stored MINC format so we load the images using `nibabel` to get the raw data later.

# Data Fetching

We now download the Brainweb brain & T1 phantoms, if they are not already present in the current directory. These are stored in MINC format, so we use 

Display the phantom data as a check

In [ ]:
if not isfile('classes.mnc'):
    print('Downloading classes')
    params = {'download_for_real':'[Start Download!]',
              'do_download_alias':'phantom_1.0mm_normal_crisp',
              'format_value':'minc',
              'who_name': 'Tobias Wood',
              'who_institution': 'KCL',
              'who_email': 'tobias.wood@kcl.ac.uk'}
    response = requests.get(url='http://brainweb.bic.mni.mcgill.ca/cgi/brainweb1', params=params)
    minc_file = open('classes.mnc', 'wb')
    minc_file.write(response.content)
if not isfile('rf20_C.mnc'):
    print('Downloading B1')
    params = {'download_for_real':'[Start Download!]',
              'do_download_alias':'rf20_C.mnc.gz',
              'format_value':'minc',
              'zip_value':'none',
              'who_name': 'Tobias Wood',
              'who_institution': 'KCL',
              'who_email': 'tobias.wood@kcl.ac.uk'}
    response = requests.get(url='https://brainweb.bic.mni.mcgill.ca/cgi/brainweb1', params=params)
    b1_file = open('rf20_C.mnc', 'wb')
    b1_file.write(response.content)
classes = nanslice.Layer('classes.mnc')
b0_minc = nanslice.Layer('rf20_B.mnc')
b1_minc = nanslice.Layer('rf20_C.mnc')

In [ ]:
ns.three_plane(classes)

Now we define a function that allows us to convert the tissue classes to T1 and PD/M0 values, and use it to create our reference T1, PD and B1 images. These will be saved into the current directory. This function also allows us to subsample the images, which is useful when investigating methods that take longer to fit.

The BrainWeb phantom uses the following mapping:

0. Background
1. CSF
2. Grey Matter
3. White Matter
4. Fat
5. Muscle/Skin
6. Skin
7. Skull
8. Glial Matter
9. Connective

To keep things simple, we only specify values for CSF, GM & WM, and set the other tissue types to zero. The B1 data is used as is from the phantom.

In [ ]:
def classes_to_params(M0Vals, T1Vals, subsamp=1):
    class_data = classes.image.get_data().astype('int32')
    M0data = np.choose(class_data[::subsamp,::subsamp,::subsamp], M0vals).astype('float32')
    T1data = np.choose(class_data[::subsamp,::subsamp,::subsamp], T1vals).astype('float32')
    T2data = np.choose(class_data[::subsamp,::subsamp,::subsamp], T2vals).astype('float32')
    B1data = b1_minc.image.get_data().astype('float32')[::subsamp,::subsamp,::subsamp]
    # PDdata = np.array(list(map(PDFunc, classes.image.get_data())))
    M0image = nib.nifti1.Nifti1Image(M0data, affine=classes.image.affine)
    T1image = nib.nifti1.Nifti1Image(T1data, affine=classes.image.affine)
    B1image = nib.nifti1.Nifti1Image(B1data, affine=classes.image.affine)
    nib.save(M0image, 'M0.nii.gz')
    nib.save(T1image, 'T1.nii.gz')
    nib.save(B1image, 'B1.nii.gz')

M0vals = np.array([0, 1, 0.8, 0.7, 0, 0, 0, 0, 0, 0])
T1vals = np.array([0, 3.0, 1.3, 0.9, 0, 0, 0, 0, 0, 0])
classes_to_params(M0vals, T1vals, 1)

## Simulate Image

As well as fitting image data to find parameters, QUIT allows you to simulate the images from the paramters. Here we use the phantom images from the step above to create some simple gradient-echo (also called FLASH, SPGR or FFE depending on vendor) images.

First we define a dictionary that sets the sequence parameters for the type of scan we are simulating. In this case we only need the TR and the flip-angle (FA). The TR is specified in seconds, not milliseconds, and there are multiple values for the flip-angle as we need to simulate multiple images.

We then define and run the simulator. This will return a results object that contains the paths to the output files, which we use to display both volumes in the output image. The noise is defined as an SNR level relative assuming a nominal PD of 1 (this is why the PD of CSF was set to 1 above).

In [ ]:
sequence_parameters = {'SPGR': {'TR': 10e-3, 'FA': [3, 18]}}
simulator_results = DESPOT1Sim(sequence=d1seq, in_file='sim_spgr.nii.gz', noise=0.001, PD='PD.nii.gz', T1='T1.nii.gz').run()

simulated_spgr = Layer(simulator_results.outputs.out_file, volume=0)
display(ns.three_plane(spgr))
spgr.volume = 1
display(ns.three_plane(spgr))

## Fit Data and Compare to Reference

Now we have simulated data we can run the DESPOT1/VFA fitting code and compare the results to our phantom reference data.

## B1 Mapping

SPGR images are affected by B1 inhomogeneity. In the above simulation, we did not specify a B1 map and assumed that B1 was flat.

In [ ]:
fitting_results = DESPOT1(sequence=d1seq, in_file='sim_spgr.nii.gz').run()

display(ns.compare('T1.nii.gz', fitting_results.outputs.t1_map, title='T1 Comparison'))
display(ns.compare('PD.nii.gz', fitting_results.outputs.pd_map, title='PD Comparison'))